In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time
import statsmodels.formula.api as sm

In [12]:
df = pd.read_csv('final_race_df.csv')
#df = pd.read_csv('final_bike_df.csv')

df2 = pd.read_csv('workouts_final_race_df.csv')
#df2 = pd.read_csv('workouts_final_bike_df.csv')

In [13]:
df['sleep_duration'] = df['rem_sleep_duration'] + df['slow_wave_sleep_duration'] + df['light_sleep_duration']

In [14]:
reg_df = pd.merge(df, df2, how='left')

In [15]:
reg_df = reg_df.drop(df.columns[0], axis=1)
reg_df.columns

Index([u'user_id', u'race_period', u'resting_heart_rate', u'hrv_rmssd', u'time_in_bed', u'latency', u'rem_sleep_duration', u'slow_wave_sleep_duration', u'light_sleep_duration', u'wake_duration', u'cycles_count', u'score', u'recovery_score', u'user_2439', u'user_2456', u'user_2458', u'user_2465', u'user_2466', u'user_2468', u'user_2469', u'user_2473', u'user_2508', u'seconds', u'FP_5K', u'Wisco_8K', u'Brown_8K', u'VCP_8K', u'FP_10K', u'sleep_duration', u'intensity_score', u'z1', u'z2', u'z3',
       u'z4', u'z5'],
      dtype='object')

In [16]:
reg_df.head()

,user_id,race_period,resting_heart_rate,hrv_rmssd,time_in_bed,latency,rem_sleep_duration,slow_wave_sleep_duration,light_sleep_duration,wake_duration,cycles_count,score,recovery_score,user_2439,user_2456,user_2458,user_2465,user_2466,user_2468,user_2469,user_2473,user_2508,seconds,FP_5K,Wisco_8K,Brown_8K,VCP_8K,FP_10K,sleep_duration,intensity_score,z1,z2,z3,z4,z5
0,828,1,45.180328,0.059197,32983054.4242,1609090.75758,4505000.00000,3321818.18182,20104090.9091,5053636.36364,7.000000,80.984848,65.426230,0,0,0,0,0,0,0,0,0,1529.2,0,0,0,0,0,27930909.09092,14.364654,580.939394,852.590909,982.393939,1601.742424,321.606061
1,828,2,47.145833,0.055524,35072882.5625,1245231.19792,3633750.00000,4436562.50000,21996562.5000,5001874.94792,5.437500,91.906250,63.802083,0,0,0,0,0,0,0,0,0,1552.3,0,0,1,0,0,30066875.00000,14.525000,725.676190,593.161905,877.771429,1497.019048,352.838095
2,828,3,45.916667,0.074007,35219680.7292,1261142.54167,1525625.00000,7692187.50000,19688750.0000,6312812.50000,3.364583,87.156250,56.385417,0,0,0,0,0,0,0,0,0,1582.5,0,0,0,1,0,28906562.50000,18.333864,876.066667,916.114286,664.714286,1752.895238,260.047619
3,828,4,44.663366,0.078605,42391598.3267,1041284.36634,3231386.13861,8411584.15842,21082277.2277,9663564.35644,6.584158,96.257426,70.940594,0,0,0,0,0,0,0,0,0,1932.7,0,0,0,0,1,32725247.52473,15.449945,570.123810,516.809524,864.561905,1844.876190,216.857143
4,828,5,45.400000,0.066477,35584066.6000,1155980.93333,2920000.00000,7196000.00000,19642000.0000,5788000.00000,5.333333,92.866667,58.733333,0,0,0,0,0,0,0,0,0,1617.6,0,0,0,1,0,29758000.00000,7.961832,319.138889,320.833333,339.777778,834.638889,61.861111


In [18]:
result1 = sm.ols(formula="seconds~ hrv_rmssd + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result2 = sm.ols(formula="seconds~ resting_heart_rate + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result3 = sm.ols(formula="seconds~ sleep_duration + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result4 = sm.ols(formula="seconds~ rem_sleep_duration + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result5 = sm.ols(formula="seconds~ slow_wave_sleep_duration + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result6 = sm.ols(formula="seconds~ light_sleep_duration + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result7 = sm.ols(formula="seconds~ time_in_bed + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result8 = sm.ols(formula="seconds~ cycles_count + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result9 = sm.ols(formula="seconds~ time_in_bed + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result10 = sm.ols(formula="seconds~ latency + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result11 = sm.ols(formula="seconds~ score + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result12 = sm.ols(formula="seconds~ recovery_score + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result13 = sm.ols(formula="seconds~ intensity_score + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result14 = sm.ols(formula="seconds~ z1 + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result15 = sm.ols(formula="seconds~ z2 + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result16 = sm.ols(formula="seconds~ z3 + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result17 = sm.ols(formula="seconds~ z4 + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()
result18 = sm.ols(formula="seconds~ z5 + user_2456 + user_2458 + user_2465 + user_2466 + user_2468 + user_2469 + user_2473 + user_2508 + FP_5K + Wisco_8K + Brown_8K + VCP_8K + FP_10K", data=reg_df).fit()


In [19]:
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     118.1
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.00e-16
Time:                        20:38:20   Log-Likelihood:                -158.95
No. Observations:                  36   AIC:                             347.9
Df Residuals:                      21   BIC:                             371.6
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1539.8726     32.677     47.124      0.000      1471.916  1607.829
hrv_rmssd   -214.5063    470.778     -0.456      0.653     -1193.543   764.530
user_2456    -41.6944     18.361     -2.271      0.034       -79.878    -3.511
user_2458     65.5947     37.300      1.759      0.093       -11.976   143.165
user_2465    -29.8553     24.207     -1.233      0.231       -80.196    20.486
user_2466     49.0151     38.865      1.261      0.221       -31.810   129.840
user_2468    -33.2016     17.406     -1.908      0.070       -69.398     2.995
user_2469     11.0848     21.701      0.511      0.615       -34.045    56.214
user_2473    -57.6303     31.641     -1.821      0.083      -123.432     8.171
user_2508     77.1230     29.479      2.616      0.016        15.817   138.429
FP_5K       -652.3537     38.876    -16.780      0.000      -733.202  -571.506
Wisco_8K      11.9941     15.879      0.755      0.458       -21.027    45.016
Brown_8K      -7.4677     22.819     -0.327      0.747       -54.922    39.987
VCP_8K        83.4421     12.236      6.819      0.000        57.996   108.888
FP_10K       427.1248     16.569     25.779      0.000       392.668   461.581
==============================================================================
Omnibus:                        1.200   Durbin-Watson:                   2.599
Prob(Omnibus):                  0.549   Jarque-Bera (JB):                0.446
Skew:                           0.217   Prob(JB):                        0.800
Kurtosis:                       3.331   Cond. No.                         127.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     118.3
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           9.89e-17
Time:                        20:38:28   Log-Likelihood:                -158.92
No. Observations:                  36   AIC:                             347.8
Df Residuals:                      21   BIC:                             371.6
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept           1443.0403    173.560      8.314      0.000      1082.102  1803.979
resting_heart_rate     1.8270      3.779      0.483      0.634        -6.033     9.687
user_2456            -43.1666     17.526     -2.463      0.022       -79.613    -6.720
user_2458             58.5891     24.910      2.352      0.029         6.787   110.392
user_2465            -42.3834     19.995     -2.120      0.046       -83.966    -0.801
user_2466             35.3723     17.914      1.975      0.062        -1.881    72.626
user_2468            -26.7740     20.956     -1.278      0.215       -70.354    16.806
user_2469             -7.0089     49.813     -0.141      0.889      -110.600    96.582
user_2473            -64.0312     22.437     -2.854      0.010      -110.692   -17.370
user_2508             71.3662     31.510      2.265      0.034         5.838   136.895
FP_5K               -662.1211     43.160    -15.341      0.000      -751.877  -572.365
Wisco_8K               9.9939     15.524      0.644      0.527       -22.290    42.278
Brown_8K              -8.6789     23.079     -0.376      0.711       -56.675    39.317
VCP_8K                82.7441     11.845      6.985      0.000        58.111   107.377
FP_10K               425.1823     15.537     27.366      0.000       392.872   457.493
==============================================================================
Omnibus:                        2.607   Durbin-Watson:                   2.570
Prob(Omnibus):                  0.272   Jarque-Bera (JB):                1.441
Skew:                           0.376   Prob(JB):                        0.486
Kurtosis:                       3.630   Cond. No.                     1.93e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.93e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     119.4
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           8.99e-17
Time:                        20:38:31   Log-Likelihood:                -158.76
No. Observations:                  36   AIC:                             347.5
Df Residuals:                      21   BIC:                             371.3
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept       1465.9206     93.690     15.647      0.000      1271.082  1660.759
sleep_duration  2.003e-06   3.05e-06      0.656      0.519     -4.34e-06  8.35e-06
user_2456        -36.6050     20.985     -1.744      0.096       -80.246     7.036
user_2458         51.7317     19.472      2.657      0.015        11.237    92.227
user_2465        -30.1087     20.640     -1.459      0.159       -73.033    12.816
user_2466         43.3114     23.159      1.870      0.075        -4.850    91.473
user_2468        -31.1224     17.365     -1.792      0.088       -67.234     4.989
user_2469         14.9547     19.750      0.757      0.457       -26.118    56.027
user_2473        -59.0015     24.996     -2.360      0.028      -110.984    -7.019
user_2508         88.8218     34.584      2.568      0.018        16.900   160.744
FP_5K           -671.2691     47.621    -14.096      0.000      -770.303  -572.235
Wisco_8K           7.6332     16.006      0.477      0.638       -25.653    40.920
Brown_8K          -6.3660     22.670     -0.281      0.782       -53.510    40.778
VCP_8K            84.8543     12.528      6.773      0.000        58.801   110.908
FP_10K           422.1420     15.770     26.769      0.000       389.347   454.937
==============================================================================
Omnibus:                        0.883   Durbin-Watson:                   2.524
Prob(Omnibus):                  0.643   Jarque-Bera (JB):                0.301
Skew:                           0.199   Prob(JB):                        0.860
Kurtosis:                       3.204   Cond. No.                     6.76e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.76e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     127.4
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           4.60e-17
Time:                        20:38:34   Log-Likelihood:                -157.60
No. Observations:                  36   AIC:                             345.2
Df Residuals:                      21   BIC:                             369.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Intercept           1543.8676     19.163     80.563      0.000      1504.015  1583.720
rem_sleep_duration  -3.14e-06   2.31e-06     -1.361      0.188     -7.94e-06  1.66e-06
user_2456            -42.5503     16.755     -2.540      0.019       -77.395    -7.706
user_2458             70.5181     23.618      2.986      0.007        21.401   119.635
user_2465            -34.6369     16.834     -2.058      0.052       -69.645     0.371
user_2466             45.3570     18.948      2.394      0.026         5.953    84.761
user_2468            -29.3650     16.852     -1.742      0.096       -64.412     5.682
user_2469             12.7666     19.200      0.665      0.513       -27.162    52.695
user_2473            -53.5609     22.466     -2.384      0.027      -100.281    -6.841
user_2508             77.6792     28.397      2.735      0.012        18.625   136.734
FP_5K               -667.0967     38.830    -17.180      0.000      -747.848  -586.345
Wisco_8K               3.6176     15.756      0.230      0.821       -29.148    36.383
Brown_8K              -6.9624     21.942     -0.317      0.754       -52.594    38.669
VCP_8K                69.4484     14.456      4.804      0.000        39.386    99.511
FP_10K               412.6830     17.212     23.976      0.000       376.889   448.477
==============================================================================
Omnibus:                        1.831   Durbin-Watson:                   2.755
Prob(Omnibus):                  0.400   Jarque-Bera (JB):                0.994
Skew:                           0.383   Prob(JB):                        0.608
Kurtosis:                       3.276   Cond. No.                     6.47e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.47e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
result5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     123.6
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           6.28e-17
Time:                        20:38:35   Log-Likelihood:                -158.14
No. Observations:                  36   AIC:                             346.3
Df Residuals:                      21   BIC:                             370.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept                 1549.2176     25.415     60.957      0.000      1496.364  1602.071
slow_wave_sleep_duration -4.429e-06   4.08e-06     -1.086      0.290     -1.29e-05  4.05e-06
user_2456                  -53.7986     19.008     -2.830      0.010       -93.328   -14.269
user_2458                   50.4563     19.129      2.638      0.015        10.675    90.237
user_2465                  -36.1379     16.999     -2.126      0.046       -71.489    -0.787
user_2466                   16.5772     22.817      0.727      0.476       -30.874    64.028
user_2468                  -33.1750     16.961     -1.956      0.064       -68.447     2.097
user_2469                    4.4426     21.744      0.204      0.840       -40.777    49.662
user_2473                  -76.5636     21.093     -3.630      0.002      -120.429   -32.699
user_2508                   65.5815     30.607      2.143      0.044         1.930   129.233
FP_5K                     -652.6024     37.991    -17.178      0.000      -731.610  -573.595
Wisco_8K                    16.0290     16.014      1.001      0.328       -17.274    49.332
Brown_8K                   -10.6634     22.550     -0.473      0.641       -57.558    36.231
VCP_8K                      90.3661     13.892      6.505      0.000        61.477   119.255
FP_10K                     435.7100     18.355     23.739      0.000       397.540   473.880
==============================================================================
Omnibus:                        0.057   Durbin-Watson:                   2.810
Prob(Omnibus):                  0.972   Jarque-Bera (JB):                0.089
Skew:                           0.065   Prob(JB):                        0.956
Kurtosis:                       2.793   Cond. No.                     6.42e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.42e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
result6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     160.7
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           4.17e-18
Time:                        20:38:36   Log-Likelihood:                -153.46
No. Observations:                  36   AIC:                             336.9
Df Residuals:                      21   BIC:                             360.7
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept             1415.4672     41.916     33.769      0.000      1328.299  1502.636
light_sleep_duration  5.637e-06   2.02e-06      2.786      0.011      1.43e-06  9.85e-06
user_2456              -30.8321     15.667     -1.968      0.062       -63.413     1.749
user_2458               86.8220     21.123      4.110      0.000        42.895   130.749
user_2465               -9.5533     17.961     -0.532      0.600       -46.905    27.798
user_2466               62.5025     18.233      3.428      0.003        24.584   100.421
user_2468              -23.9744     15.194     -1.578      0.130       -55.571     7.622
user_2469               -2.9547     18.236     -0.162      0.873       -40.878    34.969
user_2473              -24.2081     23.620     -1.025      0.317       -73.328    24.911
user_2508               98.0301     26.432      3.709      0.001        43.063   152.997
FP_5K                 -729.1809     43.137    -16.904      0.000      -818.889  -639.473
Wisco_8K                -2.5806     14.112     -0.183      0.857       -31.928    26.767
Brown_8K               -10.5982     19.606     -0.541      0.594       -51.371    30.175
VCP_8K                  79.1650     10.080      7.853      0.000        58.202   100.128
FP_10K                 411.3234     14.088     29.197      0.000       382.026   440.621
==============================================================================
Omnibus:                        1.532   Durbin-Watson:                   2.921
Prob(Omnibus):                  0.465   Jarque-Bera (JB):                1.058
Skew:                          -0.085   Prob(JB):                        0.589
Kurtosis:                       2.177   Cond. No.                     3.09e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.09e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
result7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.11e-16
Time:                        20:38:38   Log-Likelihood:                -159.12
No. Observations:                  36   AIC:                             348.2
Df Residuals:                      21   BIC:                             372.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept    1523.5841     90.960     16.750      0.000      1334.423  1712.746
time_in_bed  8.573e-08   2.53e-06      0.034      0.973     -5.17e-06  5.34e-06
user_2456     -43.9729     22.479     -1.956      0.064       -90.720     2.774
user_2458      51.2628     20.087      2.552      0.019         9.490    93.036
user_2465     -37.0207     23.481     -1.577      0.130       -85.853    11.811
user_2466      33.5108     20.181      1.660      0.112        -8.458    75.480
user_2468     -32.2873     18.275     -1.767      0.092       -70.293     5.718
user_2469      15.3196     21.184      0.723      0.478       -28.735    59.374
user_2473     -68.1510     26.082     -2.613      0.016      -122.391   -13.911
user_2508      77.4945     36.324      2.133      0.045         1.955   153.034
FP_5K        -653.7254     44.384    -14.729      0.000      -746.026  -561.425
Wisco_8K       10.2389     16.637      0.615      0.545       -24.361    44.838
Brown_8K       -6.8358     22.889     -0.299      0.768       -54.436    40.765
VCP_8K         81.7809     11.748      6.961      0.000        57.349   106.212
FP_10K        424.1146     17.899     23.695      0.000       386.891   461.338
==============================================================================
Omnibus:                        1.557   Durbin-Watson:                   2.639
Prob(Omnibus):                  0.459   Jarque-Bera (JB):                0.693
Skew:                           0.281   Prob(JB):                        0.707
Kurtosis:                       3.382   Cond. No.                     7.77e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.77e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
result8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     130.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           3.73e-17
Time:                        20:38:38   Log-Likelihood:                -157.24
No. Observations:                  36   AIC:                             344.5
Df Residuals:                      21   BIC:                             368.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept     1562.0547     27.289     57.241      0.000      1505.304  1618.806
cycles_count    -5.8308      3.832     -1.522      0.143       -13.799     2.138
user_2456      -48.0481     16.698     -2.877      0.009       -82.774   -13.323
user_2458       58.7264     19.305      3.042      0.006        18.579    98.874
user_2465      -37.3475     16.531     -2.259      0.035       -71.725    -2.970
user_2466       42.5260     17.638      2.411      0.025         5.846    79.206
user_2468      -34.1045     16.565     -2.059      0.052       -68.553     0.344
user_2469        8.4642     19.426      0.436      0.667       -31.935    48.864
user_2473      -56.4269     20.940     -2.695      0.014       -99.973   -12.880
user_2508       60.2203     30.139      1.998      0.059        -2.458   122.898
FP_5K         -659.9975     37.335    -17.677      0.000      -737.641  -582.354
Wisco_8K         9.7413     14.796      0.658      0.517       -21.028    40.511
Brown_8K        -4.6156     21.772     -0.212      0.834       -49.893    40.662
VCP_8K          75.2344     11.950      6.296      0.000        50.383   100.086
FP_10K         419.8000     15.058     27.878      0.000       388.485   451.115
==============================================================================
Omnibus:                        0.595   Durbin-Watson:                   2.646
Prob(Omnibus):                  0.743   Jarque-Bera (JB):                0.203
Skew:                           0.180   Prob(JB):                        0.904
Kurtosis:                       3.074   Cond. No.                         62.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
result9.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.11e-16
Time:                        20:38:39   Log-Likelihood:                -159.12
No. Observations:                  36   AIC:                             348.2
Df Residuals:                      21   BIC:                             372.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept    1523.5841     90.960     16.750      0.000      1334.423  1712.746
time_in_bed  8.573e-08   2.53e-06      0.034      0.973     -5.17e-06  5.34e-06
user_2456     -43.9729     22.479     -1.956      0.064       -90.720     2.774
user_2458      51.2628     20.087      2.552      0.019         9.490    93.036
user_2465     -37.0207     23.481     -1.577      0.130       -85.853    11.811
user_2466      33.5108     20.181      1.660      0.112        -8.458    75.480
user_2468     -32.2873     18.275     -1.767      0.092       -70.293     5.718
user_2469      15.3196     21.184      0.723      0.478       -28.735    59.374
user_2473     -68.1510     26.082     -2.613      0.016      -122.391   -13.911
user_2508      77.4945     36.324      2.133      0.045         1.955   153.034
FP_5K        -653.7254     44.384    -14.729      0.000      -746.026  -561.425
Wisco_8K       10.2389     16.637      0.615      0.545       -24.361    44.838
Brown_8K       -6.8358     22.889     -0.299      0.768       -54.436    40.765
VCP_8K         81.7809     11.748      6.961      0.000        57.349   106.212
FP_10K        424.1146     17.899     23.695      0.000       386.891   461.338
==============================================================================
Omnibus:                        1.557   Durbin-Watson:                   2.639
Prob(Omnibus):                  0.459   Jarque-Bera (JB):                0.693
Skew:                           0.281   Prob(JB):                        0.707
Kurtosis:                       3.382   Cond. No.                     7.77e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.77e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
result10.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.6
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.05e-16
Time:                        20:38:39   Log-Likelihood:                -159.03
No. Observations:                  36   AIC:                             348.1
Df Residuals:                      21   BIC:                             371.8
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1522.2252     19.964     76.249      0.000      1480.708  1563.742
latency      4.01e-06    1.2e-05      0.333      0.742      -2.1e-05   2.9e-05
user_2456    -43.6852     17.525     -2.493      0.021       -80.130    -7.240
user_2458     50.6292     19.652      2.576      0.018         9.760    91.498
user_2465    -36.8678     17.494     -2.107      0.047       -73.249    -0.487
user_2466     32.5215     17.479      1.861      0.077        -3.828    68.871
user_2468    -34.5885     18.494     -1.870      0.075       -73.049     3.872
user_2469     16.2429     20.203      0.804      0.430       -25.772    58.257
user_2473    -68.8483     20.312     -3.390      0.003      -111.089   -26.608
user_2508     78.6978     30.092      2.615      0.016        16.117   141.278
FP_5K       -654.4572     39.180    -16.704      0.000      -735.937  -572.977
Wisco_8K      10.4983     15.543      0.675      0.507       -21.825    42.822
Brown_8K      -7.5198     22.921     -0.328      0.746       -55.186    40.146
VCP_8K        80.7164     12.150      6.643      0.000        55.450   105.983
FP_10K       423.9474     15.564     27.238      0.000       391.580   456.315
==============================================================================
Omnibus:                        2.102   Durbin-Watson:                   2.643
Prob(Omnibus):                  0.350   Jarque-Bera (JB):                1.061
Skew:                           0.332   Prob(JB):                        0.588
Kurtosis:                       3.516   Cond. No.                     1.41e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
result11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     125.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           5.60e-17
Time:                        20:38:40   Log-Likelihood:                -157.94
No. Observations:                  36   AIC:                             345.9
Df Residuals:                      21   BIC:                             369.6
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1418.9963     91.363     15.531      0.000      1228.996  1608.997
score          1.2541      1.051      1.193      0.246        -0.932     3.440
user_2456    -21.6361     25.492     -0.849      0.406       -74.649    31.377
user_2458     44.2957     19.856      2.231      0.037         3.002    85.589
user_2465    -19.5865     22.603     -0.867      0.396       -66.592    27.419
user_2466     62.2047     29.604      2.101      0.048         0.639   123.770
user_2468    -28.2082     17.231     -1.637      0.117       -64.042     7.625
user_2469     11.9114     19.488      0.611      0.548       -28.617    52.440
user_2473    -55.4493     22.619     -2.451      0.023      -102.488    -8.410
user_2508    106.3822     37.905      2.807      0.011        27.554   185.210
FP_5K       -700.3920     54.812    -12.778      0.000      -814.379  -586.405
Wisco_8K      -0.3790     17.595     -0.022      0.983       -36.969    36.211
Brown_8K     -10.7993     22.398     -0.482      0.635       -57.379    35.780
VCP_8K        78.6163     11.673      6.735      0.000        54.341   102.892
FP_10K       409.5635     19.523     20.979      0.000       368.964   450.163
==============================================================================
Omnibus:                        0.585   Durbin-Watson:                   2.410
Prob(Omnibus):                  0.746   Jarque-Bera (JB):                0.236
Skew:                           0.198   Prob(JB):                        0.889
Kurtosis:                       3.035   Cond. No.                     2.18e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
result12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.11e-16
Time:                        20:38:40   Log-Likelihood:                -159.12
No. Observations:                  36   AIC:                             348.2
Df Residuals:                      21   BIC:                             372.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept       1526.4395     41.761     36.552      0.000      1439.594  1613.285
recovery_score     0.0033      0.672      0.005      0.996        -1.394     1.400
user_2456        -44.4343     17.940     -2.477      0.022       -81.743    -7.126
user_2458         51.0706     22.241      2.296      0.032         4.817    97.324
user_2465        -37.5621     17.477     -2.149      0.043       -73.908    -1.216
user_2466         33.1835     17.837      1.860      0.077        -3.911    70.278
user_2468        -32.4820     17.476     -1.859      0.077       -68.825     3.861
user_2469         15.0593     20.313      0.741      0.467       -27.184    57.303
user_2473        -68.7366     21.443     -3.206      0.004      -113.329   -24.144
user_2508         76.8313     31.397      2.447      0.023        11.537   142.126
FP_5K           -653.1325     46.638    -14.004      0.000      -750.122  -556.144
Wisco_8K          10.3990     17.422      0.597      0.557       -25.832    46.630
Brown_8K          -6.8564     23.177     -0.296      0.770       -55.057    41.344
VCP_8K            81.7776     12.065      6.778      0.000        56.687   106.869
FP_10K           424.3784     17.361     24.445      0.000       388.275   460.482
==============================================================================
Omnibus:                        1.601   Durbin-Watson:                   2.646
Prob(Omnibus):                  0.449   Jarque-Bera (JB):                0.723
Skew:                           0.287   Prob(JB):                        0.697
Kurtosis:                       3.389   Cond. No.                         905.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
result13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.5
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.06e-16
Time:                        20:39:06   Log-Likelihood:                -159.04
No. Observations:                  36   AIC:                             348.1
Df Residuals:                      21   BIC:                             371.8
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept        1532.3183     23.447     65.353      0.000      1483.558  1581.078
intensity_score    -0.3440      1.091     -0.315      0.756        -2.613     1.925
user_2456         -42.9479     18.022     -2.383      0.027       -80.427    -5.468
user_2458          51.0744     19.603      2.605      0.017        10.308    91.841
user_2465         -38.3862     17.576     -2.184      0.040       -74.937    -1.836
user_2466          35.6835     19.125      1.866      0.076        -4.089    75.456
user_2468         -31.3729     17.725     -1.770      0.091       -68.234     5.488
user_2469          17.4440     21.272      0.820      0.421       -26.793    61.681
user_2473         -70.0892     20.782     -3.373      0.003      -113.308   -26.870
user_2508          77.6946     29.687      2.617      0.016        15.957   139.432
FP_5K            -657.7396     41.740    -15.758      0.000      -744.542  -570.937
Wisco_8K           10.8467     15.600      0.695      0.494       -21.596    43.290
Brown_8K           -5.7983     23.073     -0.251      0.804       -53.781    42.184
VCP_8K             80.1932     12.766      6.282      0.000        53.645   106.742
FP_10K            422.4158     16.752     25.215      0.000       387.577   457.255
==============================================================================
Omnibus:                        2.067   Durbin-Watson:                   2.655
Prob(Omnibus):                  0.356   Jarque-Bera (JB):                1.051
Skew:                           0.345   Prob(JB):                        0.591
Kurtosis:                       3.475   Cond. No.                         190.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
result14.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     122.3
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           7.03e-17
Time:                        20:39:12   Log-Likelihood:                -158.33
No. Observations:                  36   AIC:                             346.7
Df Residuals:                      21   BIC:                             370.4
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1542.0764     21.657     71.205      0.000      1497.038  1587.114
z1            -0.0189      0.019     -0.970      0.343        -0.059     0.022
user_2456    -42.4061     17.171     -2.470      0.022       -78.114    -6.698
user_2458     53.5854     19.390      2.764      0.012        13.262    93.908
user_2465    -39.0030     17.105     -2.280      0.033       -74.576    -3.430
user_2466     35.1270     17.160      2.047      0.053        -0.559    70.813
user_2468    -31.9789     17.048     -1.876      0.075       -67.432     3.474
user_2469     33.3426     27.119      1.229      0.232       -23.055    89.740
user_2473    -67.7032     19.946     -3.394      0.003      -109.183   -26.224
user_2508     77.7109     28.989      2.681      0.014        17.425   137.997
FP_5K       -663.7185     39.758    -16.694      0.000      -746.401  -581.036
Wisco_8K       5.7915     15.979      0.362      0.721       -27.439    39.022
Brown_8K      -6.8118     22.393     -0.304      0.764       -53.381    39.757
VCP_8K        75.4333     13.228      5.703      0.000        47.925   102.942
FP_10K       417.8220     16.655     25.087      0.000       383.186   452.458
==============================================================================
Omnibus:                        1.042   Durbin-Watson:                   2.764
Prob(Omnibus):                  0.594   Jarque-Bera (JB):                0.394
Skew:                           0.226   Prob(JB):                        0.821
Kurtosis:                       3.240   Cond. No.                     8.78e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [34]:
result15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     129.9
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           3.77e-17
Time:                        20:39:21   Log-Likelihood:                -157.26
No. Observations:                  36   AIC:                             344.5
Df Residuals:                      21   BIC:                             368.3
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1554.1071     23.068     67.370      0.000      1506.134  1602.080
z2            -0.0339      0.022     -1.514      0.145        -0.081     0.013
user_2456    -46.6090     16.599     -2.808      0.011       -81.129   -12.089
user_2458     52.4054     18.675      2.806      0.011        13.568    91.243
user_2465    -40.3782     16.643     -2.426      0.024       -74.989    -5.767
user_2466     43.6628     17.932      2.435      0.024         6.371    80.955
user_2468    -23.3547     17.600     -1.327      0.199       -59.957    13.248
user_2469     28.7084     20.972      1.369      0.186       -14.906    72.323
user_2473    -77.4332     20.173     -3.838      0.001      -119.386   -35.481
user_2508     85.8355     28.748      2.986      0.007        26.051   145.620
FP_5K       -678.3860     40.681    -16.676      0.000      -762.987  -593.785
Wisco_8K       5.4177     15.162      0.357      0.724       -26.114    36.950
Brown_8K      -6.4766     21.735     -0.298      0.769       -51.676    38.723
VCP_8K        72.2340     12.813      5.637      0.000        45.587    98.881
FP_10K       414.3754     16.177     25.614      0.000       380.732   448.018
==============================================================================
Omnibus:                        1.696   Durbin-Watson:                   2.603
Prob(Omnibus):                  0.428   Jarque-Bera (JB):                0.720
Skew:                           0.096   Prob(JB):                        0.698
Kurtosis:                       3.666   Cond. No.                     8.85e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.85e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
result16.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.7
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.04e-16
Time:                        20:39:37   Log-Likelihood:                -159.01
No. Observations:                  36   AIC:                             348.0
Df Residuals:                      21   BIC:                             371.8
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1520.5654     22.382     67.938      0.000      1474.020  1567.111
z3             0.0071      0.020      0.364      0.719        -0.034     0.048
user_2456    -42.4781     18.192     -2.335      0.030       -80.311    -4.645
user_2458     51.0377     19.588      2.606      0.017        10.302    91.774
user_2465    -39.3171     18.025     -2.181      0.041       -76.802    -1.832
user_2466     28.3079     21.896      1.293      0.210       -17.227    73.843
user_2468    -34.5342     18.261     -1.891      0.072       -72.510     3.441
user_2469     12.0047     21.605      0.556      0.584       -32.925    56.935
user_2473    -65.6464     21.965     -2.989      0.007      -111.326   -19.967
user_2508     74.2463     30.332      2.448      0.023        11.168   137.324
FP_5K       -648.6369     40.729    -15.926      0.000      -733.338  -563.936
Wisco_8K      10.0112     15.578      0.643      0.527       -22.385    42.407
Brown_8K      -9.0119     23.585     -0.382      0.706       -58.060    40.036
VCP_8K        83.7853     12.926      6.482      0.000        56.905   110.666
FP_10K       426.3711     16.397     26.003      0.000       392.272   460.471
==============================================================================
Omnibus:                        0.849   Durbin-Watson:                   2.657
Prob(Omnibus):                  0.654   Jarque-Bera (JB):                0.301
Skew:                           0.206   Prob(JB):                        0.860
Kurtosis:                       3.174   Cond. No.                     1.07e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
result17.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.3
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.07e-16
Time:                        20:39:47   Log-Likelihood:                -159.07
No. Observations:                  36   AIC:                             348.1
Df Residuals:                      21   BIC:                             371.9
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1531.9412     25.540     59.982      0.000      1478.828  1585.055
z4            -0.0030      0.012     -0.257      0.800        -0.027     0.021
user_2456    -45.6864     18.039     -2.533      0.019       -83.201    -8.172
user_2458     51.1522     19.618      2.607      0.016        10.354    91.950
user_2465    -39.2562     18.609     -2.110      0.047       -77.956    -0.556
user_2466     34.8310     18.561      1.877      0.075        -3.769    73.431
user_2468    -32.2698     17.409     -1.854      0.078       -68.474     3.934
user_2469     15.2788     19.935      0.766      0.452       -26.178    56.735
user_2473    -70.0217     20.966     -3.340      0.003      -113.623   -26.420
user_2508     77.0418     29.586      2.604      0.017        15.514   138.570
FP_5K       -658.0975     43.726    -15.050      0.000      -749.031  -567.164
Wisco_8K      10.5032     15.561      0.675      0.507       -21.857    42.863
Brown_8K      -6.0043     23.083     -0.260      0.797       -54.009    42.000
VCP_8K        80.2383     13.193      6.082      0.000        52.802   107.674
FP_10K       422.7840     16.768     25.214      0.000       387.914   457.654
==============================================================================
Omnibus:                        2.215   Durbin-Watson:                   2.648
Prob(Omnibus):                  0.330   Jarque-Bera (JB):                1.151
Skew:                           0.353   Prob(JB):                        0.562
Kurtosis:                       3.519   Cond. No.                     1.72e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.72e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [38]:
result18.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                seconds   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     117.0
Date:                Sun, 17 Jan 2016   Prob (F-statistic):           1.11e-16
Time:                        20:39:59   Log-Likelihood:                -159.12
No. Observations:                  36   AIC:                             348.2
Df Residuals:                      21   BIC:                             372.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1526.9884     15.960     95.677      0.000      1493.798  1560.179
z5            -0.0011      0.017     -0.066      0.948        -0.037     0.034
user_2456    -43.0967     26.936     -1.600      0.125       -99.113    12.920
user_2458     50.8891     19.957      2.550      0.019         9.385    92.393
user_2465    -37.7489     17.661     -2.137      0.044       -74.477    -1.021
user_2466     33.2671     17.485      1.903      0.071        -3.094    69.628
user_2468    -32.5111     17.425     -1.866      0.076       -68.748     3.725
user_2469     15.0436     19.955      0.754      0.459       -26.456    56.543
user_2473    -69.0126     20.887     -3.304      0.003      -112.449   -25.577
user_2508     76.7258     29.624      2.590      0.017        15.120   138.332
FP_5K       -653.1736     39.117    -16.698      0.000      -734.521  -571.826
Wisco_8K      10.7309     16.204      0.662      0.515       -22.968    44.430
Brown_8K      -6.7759     22.907     -0.296      0.770       -54.413    40.861
VCP_8K        81.7001     11.824      6.910      0.000        57.111   106.289
FP_10K       424.1024     16.247     26.103      0.000       390.314   457.891
==============================================================================
Omnibus:                        1.666   Durbin-Watson:                   2.642
Prob(Omnibus):                  0.435   Jarque-Bera (JB):                0.768
Skew:                           0.296   Prob(JB):                        0.681
Kurtosis:                       3.401   Cond. No.                     6.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""